In [612]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import seaborn as sns
import re
from os import path
from glob import glob
from tqdm import tqdm
import numpy as np
from matplotlib import cm
from pprint import pprint

In [613]:
ROOT_PATH = '\\\\CILab_NAS_Data/homes/inchang/PuzzlePCG/ProxyPlayer'

In [614]:
files = glob(path.join(ROOT_PATH, '*', 'MatchResult_*.csv'))


In [615]:
StatisticsColumns = [
    'ReachedKnowledgeStep',
    #'AlmostReachedKnowledgeStep',
    'Matched_HorizontalPiece', 'Matched_VerticalPiece', 'Matched_CrossPiece', 'Matched_RocketPiece', 'Matched_BombPiece', 'Matched_RainbowPiece']

In [616]:
replacements = {
    'met': 'method',
    'pla': 'player',
    'eff': 'simpleEffect',
    'obj': 'objective',
    'sim': 'simulation',
    'rat': 'almostRatio',
    'pdep': 'playerDepth',
}

def parse_filename(data):
    result = dict()
    for i in range(0, len(data), 2):
        if data[i] in replacements.keys():
            data[i] = replacements[data[i]]

        result[data[i]] = data[i+1]

    return result

In [617]:
dfs = []
for file_path in tqdm(files):
    # print(file_path)
    file_name = path.basename(file_path)

    _file_name = file_name.split('_')[1:-1]
    args = parse_filename(_file_name)

    df = pd.read_csv(file_path)

    for key, value in args.items():
        df[key] = value
    dfs.append(df)

original_df = pd.concat(dfs)

original_df['Sum_Matched'] = original_df['Matched_HorizontalPiece'] + original_df['Matched_HorizontalPiece'] + original_df['Matched_HorizontalPiece']  + \
    original_df['Matched_HorizontalPiece'] + original_df['Matched_HorizontalPiece'] + original_df['Matched_HorizontalPiece']
original_df.loc[original_df['method'] == 'random', 'simulation'] = 0

original_df['simpleEffect'] = original_df['simpleEffect'].replace(np.nan, 0)
original_df['objective'] = original_df['objective'].replace(np.nan, '-')

original_df.loc[original_df['method'] == 'random', 'playerDepth'] = '-'
original_df.loc[original_df['method'] == 'random', 'simulation'] = '-'
original_df.loc[original_df['method'] == 'random', 'almostRatio'] = '-'
original_df.loc[original_df['objective'] == 'kp', 'almostRatio'] = '-'
original_df.loc[original_df['objective'] == 'score', 'almostRatio'] = '-'

original_df.to_csv('original_df.csv')

100%|██████████| 88/88 [00:01<00:00, 53.78it/s]


In [618]:
# original_df = pd.read_csv('original_df.csv')
print(set(original_df['objective']))

{'score', 'knowledge', '-', 'kp'}


In [619]:
set(original_df['playerDepth'])


{'-', '1'}

In [620]:
original_df.head()


,EpisodeCount,StepCount,Time,InstanceUUID,SettleCount,ChangedCount,MeanComparisonCount,StdComparisonCount,ReachedKnowledgeStep,Q1ReachedKnowledgeStep,...,Target_BombPiece,Target_RainbowPiece,method,player,simpleEffect,objective,simulation,almostRatio,playerDepth,Sum_Matched
0,1,300,2023-05-24 07:17:37.509,49bab6ff,0,339,0.909664,0.709508,74,59,...,2,3,mcts,0,1,knowledge,200,1.0,1,720
1,2,300,2023-05-24 07:20:37.269,49bab6ff,0,352,0.929019,0.702039,36,7,...,2,3,mcts,0,1,knowledge,200,1.0,1,732
2,3,300,2023-05-24 07:23:45.125,49bab6ff,0,315,0.888158,0.728231,116,69,...,2,3,mcts,0,1,knowledge,200,1.0,1,606
3,4,300,2023-05-24 07:27:19.570,49bab6ff,0,376,0.970120,0.734863,31,30,...,2,3,mcts,0,1,knowledge,200,1.0,1,930
4,5,300,2023-05-24 07:31:08.750,49bab6ff,0,389,1.010020,0.695583,172,146,...,2,3,mcts,0,1,knowledge,200,1.0,1,888


In [621]:
original_df.columns.values.tolist()

['EpisodeCount',
 'StepCount',
 'Time',
 'InstanceUUID',
 'SettleCount',
 'ChangedCount',
 'MeanComparisonCount',
 'StdComparisonCount',
 'ReachedKnowledgeStep',
 'Q1ReachedKnowledgeStep',
 'Q2ReachedKnowledgeStep',
 'Q3ReachedKnowledgeStep',
 'Matched_HorizontalPiece',
 'Matched_VerticalPiece',
 'Matched_CrossPiece',
 'Matched_RocketPiece',
 'Matched_BombPiece',
 'Matched_RainbowPiece',
 'Target_HorizontalPiece',
 'Target_VerticalPiece',
 'Target_CrossPiece',
 'Target_RocketPiece',
 'Target_BombPiece',
 'Target_RainbowPiece',
 'method',
 'player',
 'simpleEffect',
 'objective',
 'simulation',
 'almostRatio',
 'playerDepth',
 'Sum_Matched']

In [622]:
pd.set_option('display.max_rows', None)
match_result = original_df.groupby(['player', 'method', 'objective', 'playerDepth', 'almostRatio', 'simulation'])[
                                      'method',
                                      'Matched_HorizontalPiece',
                                      'Matched_VerticalPiece',
                                      'Matched_CrossPiece',
                                      'Matched_RocketPiece',
                                      'Matched_BombPiece',
                                      'Matched_RainbowPiece',
                                      #'AlmostReachedKnowledgeStep',
                                      'Q1ReachedKnowledgeStep',
                                      'Q2ReachedKnowledgeStep',
                                      'Q3ReachedKnowledgeStep',
                                      'ReachedKnowledgeStep',
                                      'Sum_Matched'].agg(
    {
        'method': ['count'],
        #'AlmostReachedKnowledgeStep': ['mean'],
        'Q1ReachedKnowledgeStep': ['mean'],
        'Q2ReachedKnowledgeStep': ['mean'],
        'Q3ReachedKnowledgeStep': ['mean'],
        'ReachedKnowledgeStep': ['mean'],
        'Matched_HorizontalPiece':['mean'],
        'Matched_VerticalPiece':['mean'],
        'Matched_CrossPiece':['mean'],
        'Matched_RocketPiece':['mean'],
        'Matched_BombPiece':['mean'],
        'Matched_RainbowPiece':['mean'],
        'Sum_Matched':['mean'],
     }
)
match_result


C:\Users\GIST\.conda\envs\mlagents\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  This is separate from the ipykernel package so we can avoid doing imports until


method  \
                                                            count   
player method objective playerDepth almostRatio simulation          
0      mcts   knowledge 1           1.0         200           175   
              kp        1           -           200           173   
              score     1           -           200           174   
       random -         -           -           -            1000   
1      mcts   knowledge 1           1.0         200           173   
              kp        1           -           200           174   
              score     1           -           200           175   
       random -         -           -           -            1000   
10     mcts   knowledge 1           1.0         200           175   
              kp        1           -           200           174   
              score     1           -           200           181   
       random -         -           -           -            1000   
2      mcts   knowledge 1           1.0         200           175   
              kp        1           -           200           176   
              score     1           -           200           173   
       random -         -           -           -            1000   
3      mcts   knowledge 1           1.0         200           181   
              kp        1           -           200           175   
              score     1           -           200           177   
       random -         -           -           -            1000   
4      mcts   knowledge 1           1.0         200           175   
              kp        1           -           200           175   
              score     1           -           200           175   
       random -         -           -           -            1000   
5      mcts   knowledge 1           1.0         200           174   
              kp        1           -           200           170   
              score     1           -           200           165   
       random -         -           -           -            1000   
6      mcts   knowledge 1           1.0         200           171   
              kp        1           -           200           173   
              score     1           -           200           175   
       random -         -           -           -            1000   
7      mcts   knowledge 1           1.0         200           173   
              kp        1           -           200           172   
              score     1           -           200           174   
       random -         -           -           -            1000   
8      mcts   knowledge 1           1.0         200           169   
              kp        1           -           200           173   
              score     1           -           200           177   
       random -         -           -           -            1000   
9      mcts   knowledge 1           1.0         200           172   
              kp        1           -           200           175   
              score     1           -           200           176   
       random -         -           -           -            1000   

                                                           Q1ReachedKnowledgeStep  \
                                                                             mean   
player method objective playerDepth almostRatio simulation                          
0      mcts   knowledge 1           1.0         200                     45.605714   
              kp        1           -           200                     44.208092   
              score     1           -           200                     42.534483   
       random -         -           -           -                       67.843000   
1      mcts   knowledge 1           1.0         200                     48.046243   
              kp        1           -           200                     51.885057   
              score     1           -           20

In [623]:
original_df = original_df[
    (original_df['ReachedKnowledgeStep'] != -1) &
    (original_df['Q1ReachedKnowledgeStep'] != -1) &
    (original_df['Q2ReachedKnowledgeStep'] != -1) &
    (original_df['Q3ReachedKnowledgeStep'] != -1)
]

for i in range(5):
    original_df['playerDepth'] = original_df['playerDepth'].replace(str(i), i)
for i in range(100, 1000, 100):
    original_df['simulation'] = original_df['simulation'].replace(str(i), i)

In [624]:
pd.set_option('display.max_rows', None)
match_result = original_df.groupby(['player', 'method', 'objective', 'playerDepth', 'almostRatio', 'simulation'])[
                                      'method',
                                      'Matched_HorizontalPiece',
                                      'Matched_VerticalPiece',
                                      'Matched_CrossPiece',
                                      'Matched_RocketPiece',
                                      'Matched_BombPiece',
                                      'Matched_RainbowPiece',
                                      #'AlmostReachedKnowledgeStep',
                                      'Q1ReachedKnowledgeStep',
                                      'Q2ReachedKnowledgeStep',
                                      'Q3ReachedKnowledgeStep',
                                      'ReachedKnowledgeStep',
                                      'Sum_Matched'].agg(
    {
        'method': ['count'],
        #'AlmostReachedKnowledgeStep': ['mean'],
        'Q1ReachedKnowledgeStep': ['mean'],
        'Q2ReachedKnowledgeStep': ['mean'],
        'Q3ReachedKnowledgeStep': ['mean'],
        'ReachedKnowledgeStep': ['mean'],
        'Matched_HorizontalPiece':['mean'],
        'Matched_VerticalPiece':['mean'],
        'Matched_CrossPiece':['mean'],
        'Matched_RocketPiece':['mean'],
        'Matched_BombPiece':['mean'],
        'Matched_RainbowPiece':['mean'],
        'Sum_Matched':['mean'],
     }
)
match_result


C:\Users\GIST\.conda\envs\mlagents\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  This is separate from the ipykernel package so we can avoid doing imports until


method  \
                                                            count   
player method objective playerDepth almostRatio simulation          
0      mcts   knowledge 1           1.0         200           175   
              kp        1           -           200           173   
              score     1           -           200           174   
       random -         -           -           -             971   
1      mcts   knowledge 1           1.0         200           173   
              kp        1           -           200           173   
              score     1           -           200           175   
       random -         -           -           -             863   
10     mcts   knowledge 1           1.0         200           175   
              kp        1           -           200           174   
              score     1           -           200           181   
       random -         -           -           -             982   
2      mcts   knowledge 1           1.0         200           174   
              kp        1           -           200           174   
              score     1           -           200           172   
       random -         -           -           -             921   
3      mcts   knowledge 1           1.0         200           180   
              kp        1           -           200           175   
              score     1           -           200           177   
       random -         -           -           -             928   
4      mcts   knowledge 1           1.0         200           175   
              kp        1           -           200           175   
              score     1           -           200           175   
       random -         -           -           -             953   
5      mcts   knowledge 1           1.0         200           174   
              kp        1           -           200           170   
              score     1           -           200           165   
       random -         -           -           -             961   
6      mcts   knowledge 1           1.0         200           170   
              kp        1           -           200           173   
              score     1           -           200           175   
       random -         -           -           -             957   
7      mcts   knowledge 1           1.0         200           173   
              kp        1           -           200           171   
              score     1           -           200           174   
       random -         -           -           -             945   
8      mcts   knowledge 1           1.0         200           169   
              kp        1           -           200           173   
              score     1           -           200           177   
       random -         -           -           -             980   
9      mcts   knowledge 1           1.0         200           170   
              kp        1           -           200           171   
              score     1           -           200           173   
       random -         -           -           -             634   

                                                           Q1ReachedKnowledgeStep  \
                                                                             mean   
player method objective playerDepth almostRatio simulation                          
0      mcts   knowledge 1           1.0         200                     45.605714   
              kp        1           -           200                     44.208092   
              score     1           -           200                     42.534483   
       random -         -           -           -                       66.473738   
1      mcts   knowledge 1           1.0         200                     48.046243   
              kp        1           -           200                     50.976879   
              score     1           -           20

In [625]:
pd.set_option('display.max_rows', None)


match_result = original_df.copy()
match_result = match_result[~match_result['almostRatio'].isin(['0.75'])]
match_result = match_result[~match_result['playerDepth'].isin([2])]
match_result = match_result[~match_result['simulation'].isin([100, 400])]

# print(set(match_result['playerDepth']))
pd.set_option('display.max_rows', None)
match_result = match_result.groupby(['method', 'objective', 'playerDepth', 'simulation'])[
                                      'method',
                                      'Matched_HorizontalPiece',
                                      'Matched_VerticalPiece',
                                      'Matched_CrossPiece',
                                      'Matched_RocketPiece',
                                      'Matched_BombPiece',
                                      'Matched_RainbowPiece',
                                      #'AlmostReachedKnowledgeStep',
                                      'Q1ReachedKnowledgeStep',
                                      'Q2ReachedKnowledgeStep',
                                      'Q3ReachedKnowledgeStep',
                                      'ReachedKnowledgeStep',
                                      'Sum_Matched'].agg(
    {
        'method': ['count'],
        #'AlmostReachedKnowledgeStep': ['mean'],
        'Q1ReachedKnowledgeStep': ['mean'],
        'Q2ReachedKnowledgeStep': ['mean'],
        'Q3ReachedKnowledgeStep': ['mean'],
        'ReachedKnowledgeStep': ['mean'],
        'Matched_HorizontalPiece':['mean'],
        'Matched_VerticalPiece':['mean'],
        'Matched_CrossPiece':['mean'],
        'Matched_RocketPiece':['mean'],
        'Matched_BombPiece':['mean'],
        'Matched_RainbowPiece':['mean'],
        'Sum_Matched':['mean'],
     }
)
match_result


C:\Users\GIST\.conda\envs\mlagents\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  if sys.path[0] == "":


method Q1ReachedKnowledgeStep  \
                                         count                   mean   
method objective playerDepth simulation                                 
mcts   knowledge 1           200          1908              50.060273   
       kp        1           200          1902              51.415878   
       score     1           200          1918              50.542753   
random -         -           -           10095              84.954532   

                                        Q2ReachedKnowledgeStep  \
                                                          mean   
method objective playerDepth simulation                          
mcts   knowledge 1           200                     61.210692   
       kp        1           200                     62.314932   
       score     1           200                     61.246090   
random -         -           -                      100.081228   

                                        Q3ReachedKnowledgeStep  \
                                                          mean   
method objective playerDepth simulation                          
mcts   knowledge 1           200                     79.435535   
       kp        1           200                     80.514721   
       score     1           200                     79.859750   
random -         -           -                      128.172164   

                                        ReachedKnowledgeStep  \
                                                        mean   
method objective playerDepth simulation                        
mcts   knowledge 1           200                   86.267296   
       kp        1           200                   87.003155   
       score     1           200                   86.567779   
random -         -           -                    137.444775   

                                        Matched_HorizontalPiece  \
                                                           mean   
method objective playerDepth simulation                           
mcts   knowledge 1           200                     141.209119   
       kp        1           200                     141.430599   
       score     1           200                     141.322211   
random -         -           -                        75.641902   

                                        Matched_VerticalPiece  \
                                                         mean   
method objective playerDepth simulation                         
mcts   knowledge 1           200                    99.697589   
       kp        1           200                    99.838065   
       score     1           200                    99.510428   
random -         -           -                      54.224170   

                                        Matched_CrossPiece  \
                                                      mean   
method objective playerDepth simulation                      
mcts   knowledge 1           200                 20.285639   
       kp        1           200                 20.226078   
       score     1           200                 20.163712   
random -         -           -                   10.406538   

                                        Matched_RocketPiece Matched_BombPiece  \
                                                       mean              mean   
method objective playerDepth simulation                                         
mcts   knowledge 1           200                 218.335954         17.308700   
       kp        1           200                 218.229758         17.520505   
       score     1           200                 218.142857         17.389468   
random -         -           -                   135.102823         10.282912   

                                        Matched_RainbowPiece Sum_Matched  
                                                        mean        mean  
method objective playerDepth simulation                                   

In [626]:
def get_legend_name(x):
    if x['method'] == 'mcts':
        if x['objective'] == 'score':
            return f"{x['method']}_{x['objective']}_{x['simulation']}_{x['playerDepth']}"
        elif x['objective'] == 'knowledge':
            return f"{x['method']}_{x['objective']}_{x['almostRatio']}_{x['simulation']}_{x['playerDepth']}"
        elif x['objective'] == 'kp':
            return f"{x['method']}_{x['objective']}_{x['almostRatio']}_{x['simulation']}_{x['playerDepth']}"
    else:
        return f"{x['method']}"

melted_df = original_df.copy()

melted_df = melted_df.groupby(['method', 'objective', 'playerDepth', 'simulation', 'player']).sample(n=100, random_state=1)
melted_df['concat_method'] = melted_df.agg(lambda x: get_legend_name(x), axis=1)
pprint(set(melted_df['concat_method']))
# Display Target
melted_df = melted_df[melted_df['concat_method'].isin(['random',
                                                       'mcts_knowledge_1.0_400_1',
                                                       'mcts_knowledge_0.75_400_1',
                                                       'mcts_knowledge_1.0_400_2',
                                                       'mcts_knowledge_0.75_400_2',
                                                       'mcts_score_400_2'])]

plt.figure(figsize=(7, 5))
melted_df = pd.melt(melted_df, id_vars=['EpisodeCount', 'objective', 'concat_method', 'simulation', 'player'],
                    value_vars=[*StatisticsColumns], ignore_index=False)

melted_df['x'] = melted_df['variable']
# melted_df['x'] = melted_df['x'].str.replace('AlmostReachedKnowledgeStep', 'ar_step')	125.0
melted_df['x'] = melted_df['x'].str.replace('ReachedKnowledgeStep', 'r_step')

melted_df['x'] = melted_df['x'].str.replace('Matched_HorizontalPiece', 'b0')
melted_df['x'] = melted_df['x'].str.replace('Matched_VerticalPiece', 'b1')
melted_df['x'] = melted_df['x'].str.replace('Matched_CrossPiece', 'b2')
melted_df['x'] = melted_df['x'].str.replace('Matched_RocketPiece', 'b3')
melted_df['x'] = melted_df['x'].str.replace('Matched_BombPiece', 'b4')
melted_df['x'] = melted_df['x'].str.replace('Matched_RainbowPiece', 'b5')

ax = sns.boxplot(x="x", y="value", hue="player", data=melted_df)
plt.xticks(rotation=45)

In [627]:
for player_no in range(0, 4):
    player_df = original_df[original_df['player'] == player_no]
    melted_df = player_df.copy()

    melted_df.rename(columns={
        'ReachedKnowledgeStep':'100',
        'Q1ReachedKnowledgeStep':'25',
        'Q2ReachedKnowledgeStep':'50',
        'Q3ReachedKnowledgeStep':'75'
    }, inplace=True)

    _columns = [
    '25', '50', '75', '100',
    # Matched_HorizontalPiece', 'Matched_VerticalPiece', 'Matched_CrossPiece', 'Matched_RocketPiece', 'Matched_BombPiece', 'Matched_RainbowPiece'
    ]

    # print(player_df.keys())
    sns.set_palette('Set2')
    melted_df = melted_df.groupby(['method', 'objective', 'playerDepth', 'simulation']).sample(n=400, random_state=1, replace=True)
    melted_df['concat_method'] = melted_df.agg(lambda x: get_legend_name(x), axis=1)
    melted_df = pd.melt(melted_df, id_vars=['EpisodeCount', 'objective', 'concat_method', 'simulation', 'player'],
                        value_vars=[*_columns], ignore_index=False)
    # pprint(set(melted_df['concat_method']))
    _methods = [
        'mcts_kp_-_400_2',
        'mcts_knowledge_1.0_400_2',
        'mcts_score_400_2',
        'random'
    ]
    melted_df = melted_df[melted_df['concat_method'].isin(_methods)]

    plt.figure(figsize=(5, 4))
    plt.clf()
    ax = sns.boxplot(x="variable", y="value", hue="concat_method", data=melted_df, dodge=True, linewidth=0.8)

    plt.legend(loc='upper left')
    ax.set_title(f'Required Steps for Learning Skill (Player: {player_no})')
    ax.set_xlabel('Progress (%)')
    ax.set_ylabel('Step')
    plt.savefig(f'./figures/learning_steps_p{player_no}', dpi=300)
    plt.show()

ValueError: No objects to concatenate

In [ ]:
player_df = original_df.copy()
melted_df = player_df.copy()

melted_df.rename(columns={
    'ReachedKnowledgeStep':'100',
    'Q1ReachedKnowledgeStep':'25',
    'Q2ReachedKnowledgeStep':'50',
    'Q3ReachedKnowledgeStep':'75'
}, inplace=True)

_columns = [
'25', '50', '75', '100',
# Matched_HorizontalPiece', 'Matched_VerticalPiece', 'Matched_CrossPiece', 'Matched_RocketPiece', 'Matched_BombPiece', 'Matched_RainbowPiece'
]

# print(player_df.keys())
sns.set_palette('Set2')
melted_df = melted_df.groupby(['method', 'objective', 'playerDepth', 'simulation']).sample(n=400, random_state=1, replace=True)
melted_df['concat_method'] = melted_df.agg(lambda x: get_legend_name(x), axis=1)
melted_df = pd.melt(melted_df, id_vars=['EpisodeCount', 'objective', 'concat_method', 'simulation', 'player'],
                    value_vars=[*_columns], ignore_index=False)
# pprint(set(melted_df['concat_method']))
_methods = [
    'mcts_kp_-_400_2',
    'mcts_knowledge_1.0_400_2',
    'mcts_score_400_2',
    'random'
]
melted_df = melted_df[melted_df['concat_method'].isin(_methods)]

plt.figure(figsize=(5, 4))
plt.clf()
ax = sns.boxplot(x="variable", y="value", hue="concat_method", data=melted_df, dodge=True, linewidth=0.8)

plt.legend(loc='upper left')
ax.set_title(f'Required Steps for Learning Skill (Player: All)')
ax.set_xlabel('Progress (%)')
ax.set_ylabel('Step')
plt.savefig(f'./figures/learning_steps_pA', dpi=300)
plt.show()



In [ ]:
for player_no in range(0, 4):
    player_df = original_df[original_df['player'] == player_no]
    melted_df = player_df.copy()

    melted_df.rename(columns={
        'ReachedKnowledgeStep':'100',
        'Q1ReachedKnowledgeStep':'25',
        'Q2ReachedKnowledgeStep':'50',
        'Q3ReachedKnowledgeStep':'75',
        'Matched_HorizontalPiece':'Horizon',
        'Matched_VerticalPiece':'Vertical',
        'Matched_CrossPiece':'Cross',
        'Matched_RocketPiece':'Rocket',
        'Matched_BombPiece':'Bomb',
        'Matched_RainbowPiece':'Rainbow',

    }, inplace=True)

    _columns = [
    # '25', '50', '75', '100',
    'Horizon', 'Vertical', 'Cross', 'Rocket', 'Bomb', 'Rainbow'
    ]

    # print(player_df.keys())
    sns.set_palette('Set3')
    melted_df = melted_df.groupby(['method', 'objective', 'playerDepth', 'simulation']).sample(n=300, random_state=1, replace=True)
    melted_df['concat_method'] = melted_df.agg(lambda x: get_legend_name(x), axis=1)
    melted_df = pd.melt(melted_df, id_vars=['EpisodeCount', 'objective', 'concat_method', 'simulation', 'player'],
                        value_vars=[*_columns], ignore_index=False)
    # pprint(set(melted_df['concat_method']))
    _methods = [
        'mcts_kp_-_400_2',
        'mcts_knowledge_1.0_400_2',
        'mcts_score_400_2',
        'random'
    ]
    melted_df = melted_df[melted_df['concat_method'].isin(_methods)]

    plt.figure(figsize=(5, 4))
    plt.clf()
    ax = sns.boxplot(x="variable", y="value", hue="concat_method", data=melted_df, linewidth=0.8)

    plt.legend(loc='upper left')
    ax.set_title(f'Matched Blocks (Player: {player_no})')
    ax.set_xlabel('Block Type')
    ax.set_ylabel('Count')
    plt.xticks(rotation=45)
    plt.savefig(f'./figures/block_counts_p{player_no}', dpi=300)
    plt.show()

In [ ]:

player_df = original_df.copy()
melted_df = player_df.copy()

melted_df.rename(columns={
    'ReachedKnowledgeStep':'100',
    'Q1ReachedKnowledgeStep':'25',
    'Q2ReachedKnowledgeStep':'50',
    'Q3ReachedKnowledgeStep':'75',
    'Matched_HorizontalPiece':'Horizon',
    'Matched_VerticalPiece':'Vertical',
    'Matched_CrossPiece':'Cross',
    'Matched_RocketPiece':'Rocket',
    'Matched_BombPiece':'Bomb',
    'Matched_RainbowPiece':'Rainbow',

}, inplace=True)

_columns = [
# '25', '50', '75', '100',
'Horizon', 'Vertical', 'Cross', 'Rocket', 'Bomb', 'Rainbow'
]

# print(player_df.keys())
sns.set_palette('Set3')
melted_df = melted_df.groupby(['method', 'objective', 'playerDepth', 'simulation']).sample(n=300, random_state=1, replace=True)
melted_df['concat_method'] = melted_df.agg(lambda x: get_legend_name(x), axis=1)
melted_df = pd.melt(melted_df, id_vars=['EpisodeCount', 'objective', 'concat_method', 'simulation', 'player'],
                    value_vars=[*_columns], ignore_index=False)
# pprint(set(melted_df['concat_method']))
_methods = [
    'mcts_kp_-_400_2',
    'mcts_knowledge_1.0_400_2',
    'mcts_score_400_2',
    'random'
]
melted_df = melted_df[melted_df['concat_method'].isin(_methods)]

plt.figure(figsize=(5, 4))
plt.clf()
ax = sns.boxplot(x="variable", y="value", hue="concat_method", data=melted_df, linewidth=0.8)

plt.legend(loc='upper left')
ax.set_title(f'Matched Blocks (Player: All)')
ax.set_xlabel('Block Type')
ax.set_ylabel('Count')
plt.xticks(rotation=45)
plt.savefig(f'./figures/block_counts_pA', dpi=300)
plt.show()

In [ ]:

for player_no in range(0, 4):
    player_df = original_df[original_df['player'] == player_no]
    melted_df = player_df.copy()

    melted_df.rename(columns={
        'ReachedKnowledgeStep':'100',
        'Q1ReachedKnowledgeStep':'25',
        'Q2ReachedKnowledgeStep':'50',
        'Q3ReachedKnowledgeStep':'75'
    }, inplace=True)

    _columns = [
    '25', '50', '75', '100',
    # Matched_HorizontalPiece', 'Matched_VerticalPiece', 'Matched_CrossPiece', 'Matched_RocketPiece', 'Matched_BombPiece', 'Matched_RainbowPiece'
    ]

    # print(player_df.keys())
    sns.set_palette('Set2')
    melted_df = melted_df.groupby(['method', 'objective', 'playerDepth', 'simulation']).sample(n=400, random_state=1, replace=True)
    melted_df['concat_method'] = melted_df.agg(lambda x: get_legend_name(x), axis=1)
    melted_df = pd.melt(melted_df, id_vars=['EpisodeCount', 'objective', 'concat_method', 'simulation', 'player'],
                        value_vars=[*_columns], ignore_index=False)
    # pprint(set(melted_df['concat_method']))
    _methods = [
        'mcts_kp_-_400_2',
        'mcts_knowledge_1.0_400_2',
        'mcts_score_400_2',
        'random'
    ]
    melted_df = melted_df[melted_df['concat_method'].isin(_methods)]

    plt.figure(figsize=(5, 4))
    plt.clf()
    ax = sns.boxplot(x="variable", y="value", hue="concat_method", data=melted_df, dodge=True, linewidth=0.8)

    plt.legend(loc='upper left')
    ax.set_title(f'Required Steps for Learning Skill (Player: {player_no})')
    ax.set_xlabel('Progress (%)')
    ax.set_ylabel('Step')
    plt.savefig(f'./figures/learning_steps_p{player_no}', dpi=300)
    plt.show()

In [ ]:

player_df = original_df.copy()
melted_df = player_df.copy()

melted_df.rename(columns={
    'ReachedKnowledgeStep':'100',
    'Q1ReachedKnowledgeStep':'25',
    'Q2ReachedKnowledgeStep':'50',
    'Q3ReachedKnowledgeStep':'75',
    'Matched_HorizontalPiece':'Horizon',
    'Matched_VerticalPiece':'Vertical',
    'Matched_CrossPiece':'Cross',
    'Matched_RocketPiece':'Rocket',
    'Matched_BombPiece':'Bomb',
    'Matched_RainbowPiece':'Rainbow',

}, inplace=True)

_columns = [
# '25', '50', '75', '100',
'Horizon', 'Vertical', 'Cross', 'Rocket', 'Bomb', 'Rainbow'
]

# print(player_df.keys())
sns.set_palette('Set3')
melted_df = melted_df.groupby(['method', 'objective', 'playerDepth', 'simulation']).sample(n=300, random_state=1, replace=True)
melted_df['concat_method'] = melted_df.agg(lambda x: get_legend_name(x), axis=1)
melted_df = pd.melt(melted_df, id_vars=['EpisodeCount', 'objective', 'concat_method', 'simulation', 'player'],
                    value_vars=[*_columns], ignore_index=False)
# pprint(set(melted_df['concat_method']))
_methods = [
    'mcts_kp_-_400_2',
    'mcts_knowledge_1.0_400_2',
    'mcts_score_400_2',
    'random'
]
melted_df = melted_df[melted_df['concat_method'].isin(_methods)]

plt.figure(figsize=(5, 4))
plt.clf()
ax = sns.boxplot(x="variable", y="value", hue="concat_method", data=melted_df, linewidth=0.8)

plt.legend(loc='upper left')
ax.set_title(f'Matched Blocks (Player: All)')
ax.set_xlabel('Block Type')
ax.set_ylabel('Count')
plt.xticks(rotation=45)
plt.savefig(f'./figures/block_counts_pA', dpi=300)
plt.show()
